In [1]:
import torch

In [3]:
import chess
import torch
from chess.polyglot import zobrist_hash
from chess import Board, Move
import random
from IPython.display import SVG, clear_output, Image
import time
import numpy as np
from numpy import float64, int64
from typing import Dict, Tuple, Sequence, Self, Optional, Any
from dataclasses import dataclass
from pathlib import Path
import sys

sys.path.append(str(Path.cwd() / ".." / "src"))
from mcts import Node, Edge, MCTS
from model_wrapper import ModelWrapper
from leela_cnn import LeelaCNN, board_to_tensor
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

In [9]:
model = LeelaCNN(10, 128)
torch.save(model.state_dict(), "leela_cnn.pth")